# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [ ]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [ ]:
def P1(num_examples=5):
### STUDENT START ###
    for i in range(num_examples):
        print 'Label' + str(i) + ':', train_labels[i], 
        print 'Data' + str(i) + ':' ,train_data[i]

### STUDENT END ###
P1(5)

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [85]:
def P2():
### STUDENT START ###
    
    # Solution to Part A
    countVectorizer = CountVectorizer()
    train_counts_default = countVectorizer.fit_transform(train_data)
    print 'Shape of vector', train_counts_default.shape
    print 'Avg number of Non zero features: ', float(train_counts_default.nnz)/2034.0
    print 'Fraction of non zero: ', float(train_counts_default.nnz)/(26879.0*2034)
    
    #Solution to Part B
    print 'First Feature String: ',countVectorizer.get_feature_names()[0]
    print 'Last Feature String: ', countVectorizer.get_feature_names()[26878]
    
    #Solution to part C
    countVectorizer2 = CountVectorizer(vocabulary=["atheism", "graphics", "space", "religion"])
    #countVectorizer.vocabulary_ = ["atheism", "graphics", "space", "religion"]
    train_counts_custom_vocab = countVectorizer2.transform(train_data)
    print 'Shape with custom vocabulary: ', train_counts_custom_vocab.shape
    print 'Avg number of Non zero features with custom vocabulary: ',float(train_counts_custom_vocab.nnz)/2034.0
    
    #Solution to part D
    countVectorizer3 = CountVectorizer(analyzer='word', ngram_range=(2,2))
    train_counts_bigram = countVectorizer3.fit_transform(train_data)
    print 'Size of Vocabulary for bigram data:' , len(countVectorizer3.vocabulary_)
    
    countVectorizer4 = CountVectorizer(analyzer='word', ngram_range=(3,3))
    train_counts_trigram = countVectorizer4.fit_transform(train_data)
    print 'Size of Vocabulary for trigram data:' , len(countVectorizer4.vocabulary_)
    
    #Solution to part E
    countVectorizer5 = CountVectorizer(min_df=10)
    train_counts_min10 = countVectorizer5.fit_transform(train_data)
    print 'Size of vocab with min_df=10: ', len(countVectorizer5.vocabulary_)
    
    #Solution to Part F
    countVectorizer6 = CountVectorizer()
    train_counts_dev = countVectorizer6.fit_transform(dev_data)
    print 'Size of vocab with dev data: ', len(countVectorizer6.vocabulary_)
    print 'Number of words missing: ', len(countVectorizer.vocabulary_)-len(countVectorizer6.vocabulary_)
    
### STUDENT END ###

P2()

Shape of vector (2034, 26879)
Avg number of Non zero features:  96.7059980334
Fraction of non zero:  0.00359782722696
First Feature String:  00
Last Feature String:  zyxel
Shape with custom vocabulary:  (2034, 4)
Avg number of Non zero features with custom vocabulary:  0.268436578171
{'atheism': 0, 'religion': 3, 'graphics': 1, 'space': 2}
Size of Vocabulary for bigram data: 194891
Size of Vocabulary for trigram data: 315692
Size of vocab with min_df=10:  3064
Size of vocab with dev data:  16246
Number of words missing:  10633


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [ ]:
def P3():
### STUDENT START ###
    countVectorizer_p3 = CountVectorizer()
    train_counts_default_p3 = countVectorizer_p3.fit_transform(train_data)
    #Creating Classifier
    knn_clf_p3 = KNeighborsClassifier()
    
    knn_params = {'n_neighbors':[1,5,10,25,75, 90,95,96,97,100]}
    gsv_knn = GridSearchCV(knn_clf_p3,knn_params,scoring='f1_weighted')
    gsv_knn.fit(train_counts_default_p3,train_labels)
    print "The best k value is: ", gsv_knn.best_params_
    
    
    mnb_clf_p3 = MultinomialNB()
    mnb_params = {'alpha':[0.000000000000001,0.00000000000001,0.0000000000001,0.000000000001,0.00000000001, 0.0000000001,0.0000001,0.00001,0.001,0.1,1,0]}
    gsv_mnb = GridSearchCV(mnb_clf_p3,mnb_params,scoring='f1_weighted')
    gsv_mnb.fit(train_counts_default_p3,train_labels)
    print "The best alpha value is: ", gsv_mnb.best_params_

    
    lr_params = [0.001,0.1,0.5,0.55,0.9]
    for c in lr_params:
        lr_clf_p3 = LogisticRegression(C=c) 
        model_p3 = lr_clf_p3.fit(train_counts_default_p3,train_labels)
        print sum(model_p3.coef_**2)
        
    ### STUDENT END ###
P3()

ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [136]:
def P4():
### STUDENT START ###
    countVectorizer_p4 = CountVectorizer()
    train_counts_default_p4 = countVectorizer_p4.fit_transform(train_data)
    dev_counts_default_p4 = countVectorizer_p4.transform(dev_data)
        
    lr_clf_p4 = LogisticRegression(C=0.5,)
    lr_clf_p4 =  lr_clf_p4.fit(train_counts_default_p4, train_labels)
    
    print 'Score:',lr_clf_p4.score(dev_counts_default_p4,dev_labels)
    
    model_coef = lr_clf_p4.coef_
    model_coef_sorted = np.argsort(model_coef, axis=1)
    
    sorted_wt_indices = model_coef_sorted[0:4,-5:].reshape(20,)
    table  = np.zeros((20,4),dtype=float)
    #print (categories)
    for row in range(len(sorted_wt_indices)):
        for column in range(4):
            table[row,column] = model_coef[column,sorted_wt_indices[row]]
    #print table
    #printTable(getFeatures(sorted_wt_indices,countVectorizer_p4.vocabulary_),table)
    
    #Bi-gram
    countVectorizer_p4_2 = CountVectorizer(analyzer='word', ngram_range=(2,2))
    train_counts_default_p4_2 = countVectorizer_p4_2.fit_transform(train_data)
    dev_counts_default_p4_2 = countVectorizer_p4_2.transform(dev_data)
    
    lr_clf_p4_2 = LogisticRegression(C=0.5,)
    lr_clf_p4_2 =  lr_clf_p4_2.fit(train_counts_default_p4_2, train_labels)
    
    print 'Bi-gram Score:',lr_clf_p4_2.score(dev_counts_default_p4_2,dev_labels)
    
    model_coef_2 = lr_clf_p4_2.coef_
    model_coef_sorted_2 = np.argsort(model_coef_2, axis=1)
    
    sorted_wt_indices_2 = model_coef_sorted_2[0:4,-5:].reshape(20,)
    table_2  = np.zeros((20,4),dtype=float)
    
    for row in range(len(sorted_wt_indices_2)):
        for column in range(4):
            table_2[row,column] = model_coef_2[column,sorted_wt_indices_2[row]]
    getFeatures(sorted_wt_indices_2,countVectorizer_p4_2.vocabulary_)
    #printTable(getFeatures(sorted_wt_indices_2,countVectorizer_p4_2.vocabulary_), table_2)
    #print getFeatures(sorted_wt_indices_2,countVectorizer_p4_2.vocabulary_)

def printTable(features,tables):
    print categories
    count=0
    print 'Printing Table with:', len(features),len(tables)
    for row in range(len(tables)):
        print features[row], tables[row]
        #print count
        #count+=1
    
    
def getFeatures(indices,vocab):
    count=0
    feature_name =[]
    for feature in vocab:
        if vocab.get(feature) in indices:
            index = (np.where(indices == vocab.get(feature))[0])[0]
            feature_name.insert(index,feature)
            #print feature_name, index
            count=count+1
        if count ==20:
            print 'Hitting Break'
            break
    print [indices[18]]
    #print feature_name
### STUDENT END ###
P4()

Score: 0.71449704142
Bi-gram Score: 0.622781065089
[u'the fbi'] 19
[u'the fbi', u'and such'] 11
[u'the fbi', u'and such', u'it was'] 10
[u'the fbi', u'and such', u'it was', u'cheers kent'] 3
[u'the fbi', u'and such', u'was just', u'it was', u'cheers kent'] 2
[u'the fbi', u'and such', u'was just', u'it was', u'cheers kent', u'but he'] 16
[u'the fbi', u'and such', u'was just', u'it was', u'cheers kent', u'but he', u'comp graphics'] 7
[u'the fbi', u'and such', u'was just', u'it was', u'cheers kent', u'but he', u'comp graphics', u'in advance'] 8
[u'the fbi', u'and such', u'was just', u'it was', u'cheers kent', u'but he', u'comp graphics', u'in advance', u'of jesus'] 15
[u'the fbi', u'and such', u'was just', u'it was', u'cheers kent', u'but he', u'comp graphics', u'in advance', u'of jesus', u'looking for'] 9
[u'the fbi', u'and such', u'was just', u'it was', u'cheers kent', u'is there', u'but he', u'comp graphics', u'in advance', u'of jesus', u'looking for'] 5
[u'the fbi', u'and such', u'was

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [81]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
### STUDENT START ###
    s= s.lower()
    s=remove_non_alpha_numeric(s)
    s=remove_number(s)
    return s

def remove_number(s):
    s = re.sub( r'\s(\d)+\s', r" ZZ " , s)
    return s

def remove_non_alpha_numeric(s):
    s=re.sub( r'([^\s\w]|_)+', r"",s)
    #s=re.search('(\w+|\s)',s).group(1)
    return s

#print train_data[0]
#print better_preprocessor(str(train_data[0]))
#print better_preprocessor(' 1234 4rf 1231 ')
### STUDENT END ###

def P5():
### STUDENT START ###
    i = 0
    countVectorizer_p5 = CountVectorizer(preprocessor=better_preprocessor)
    train_counts_default_p5 = countVectorizer_p5.fit_transform(train_data)
    
    #prep = countVectorizer_p5.build_preprocessor()
    #print str(train_data[0])
    #print prep(str(train_data[0]))
    #print better_preprocessor(str(train_data[0]))

    print len(countVectorizer_p5.vocabulary_)
    #print sorted(countVectorizer_p5.vocabulary_)
    dev_counts_default_p5 = countVectorizer_p5.transform(dev_data)
    
    lr_clf_p5 = LogisticRegression(C=0.5)
    lr_clf_p5 = lr_clf_p5.fit(train_counts_default_p5,train_labels)
    predicted_labels_p5 =  lr_clf_p5.predict(dev_counts_default_p5)
    print metrics.f1_score(dev_labels,predicted_labels_p5)
    
    
### STUDENT END ###
P5()

28295
0.690772677581


/home/divyang/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [108]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###
    countVectorizer_p6 = CountVectorizer()
    train_counts_default_p6 = countVectorizer_p6.fit_transform(train_data)
    dev_counts_default_p6 = countVectorizer_p6.transform(dev_data)
        
    lr_clf_p6 = LogisticRegression(penalty='l1',C=0.5,tol=0.01)
    lr_clf_p6 =  lr_clf_p6.fit(train_counts_default_p6, train_labels)
    
    lr_clf_p6_2 = LogisticRegression(penalty='l2',C=0.5,tol=0.01)
    lr_clf_p6_2 =  lr_clf_p6_2.fit(train_counts_default_p6, train_labels)

    print 'Number of Non Zero weights for l1: ', np.count_nonzero(lr_clf_p6.coef_)
    print 'Number of Non Zero weights for l2: ',np.count_nonzero(lr_clf_p6_2.coef_)
    
    modified_vocab = getModifiedVocabulary(lr_clf_p6.coef_, countVectorizer_p6.vocabulary_)

    countVectorizer_p6_3 = CountVectorizer(vocabulary=modified_vocab)
    train_counts_l2_p6= countVectorizer_p6_3.fit_transform(train_data)
    dev_counts_l2_p6 = countVectorizer_p6_3.transform(dev_data)
    c_values = [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
    accuracies = []
    for c in c_values:
        lr_clf_p6_3 = LogisticRegression(penalty='l2',C=c,tol=0.01)
        lr_clf_p6_3 =  lr_clf_p6_3.fit(train_counts_l2_p6, train_labels)
        predicted_labels = lr_clf_p6_3.predict(dev_counts_l2_p6)
        accuracies.append(metrics.f1_score(dev_labels,predicted_labels,average='weighted'))
    
    print accuracies
def getModifiedVocabulary(weights, vocab):
    mod_vocab = []
    count =0 
    for key in vocab:
        index = vocab.get(key)
        if weights[0,index] !=0 or weights[1,index]!=0 or weights[2,index]!=0 or weights[3,index]!=0:
            mod_vocab.append(key)
            count+=1
    return mod_vocab    
    ### STUDENT END ###
P6()

Number of Non Zero weights for l1:  1213
Number of Non Zero weights for l2:  107516
26879
(4, 26879)
[0.66012904818525708, 0.66515812104222305, 0.66524715814518698, 0.66537954909089092, 0.66717472795823463, 0.66047788332781387, 0.6649422136201002, 0.66312845749970006, 0.65552379871430688, 0.65773500127793016]


(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [112]:
def P7():
    ### STUDENT START ###
    tf_Vectorizer = TfidfVectorizer()
    train_tf_count_p7 = tf_Vectorizer.fit_transform(train_data)
    dev_tf_count_p7 = tf_Vectorizer.transform(dev_data)
    
    lr_clf_p7 = LogisticRegression(C=100)
    lr_clf_p7 = lr_clf_p7.fit(train_tf_count_p7,train_labels)
    predicted_labels_p7 = lr_clf_p7.predict(dev_tf_count_p7)
    print metrics.f1_score(dev_labels,predicted_labels_p7)


    ### STUDENT END ###
P7()

0.759766242785


/home/divyang/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance